In [1]:
from doodle.dosimetry.BaseDosimetry import BaseDosimetry
from doodle.ImagingTools.Tools import create_logitudinal_from_dicom, load_and_resample_RT
import matplotlib.pyplot as plt
from pathlib import Path

# Autoreload magic
%load_ext autoreload
%autoreload 2


In [3]:
# Prepare DataStructures for CT and NM Data
db_dir = Path("/home/esquinas/Data/PR21_dosimetry/CAVA0004/cycle01")

# Set Paths to each time point. In my example, I only have 1 time point ... :(
time_points = [db_dir]

# Create Longitudinal Studies:
ct_paths = []
ctac_paths = []
spect_paths = []

for dir in time_points:
    ct_paths.append(str(dir / "CT"))
    ctac_paths.append(str(dir / "CTAC"))
    spect_paths.append(str(dir / "SPECT"))

longCT = create_logitudinal_from_dicom(dicom_dirs=ct_paths, modality="CT")
longCTAC = create_logitudinal_from_dicom(dicom_dirs=ctac_paths, modality="CT")
longSPECT = create_logitudinal_from_dicom(dicom_dirs=spect_paths, modality="Lu177_SPECT")

# Add Masks: Here we need to read RTStruct from CT, and re-sample to CTAC (== SPECT) frame.
ct_masks = {}
ctac_masks = {}

for time_id, dir in enumerate(time_points):
    ct_, ctac_ = load_and_resample_RT(str(dir / "CT"), target_shape=longCTAC.images[time_id].GetSize())
    longCT.add_masks_to_time_point(time_id=time_id, masks=ct_)
    longCTAC.add_masks_to_time_point(time_id=time_id, masks=ctac_)
    longSPECT.add_masks_to_time_point(time_id=time_id, masks=ctac_)


In [16]:
# Configuration File
# We should look into making a yaml file (similar to pyRadiomics) to specify input configuration. For now... let's make
# it a simple Dict.
config = {
    "Operator": "PLEF",
    "PatientID": "PR21-CAVA-0004",
    "Cycle": 1,
    "DatabaseDir": "./TestDoseDB",
    "InjectionDate": "20210909",   #YYYYMMDD
    "InjectionTime": "100000", #HHMMSS in 24h
    "rois": {"Liver": {"fit_order": 1, "through_origin": False},  
             "Kidney_L_m": {"fit_order": 1, "through_origin": False},  # 1 for mono, 2 for bi, 3 for tri-exponential
             "Kidney_R_m": {"fit_order": 1, "through_origin": False}, 
             "BoneMarrow": {"fit_order": 1, "through_origin": False}, 
             "Spleen": {"fit_order": 1, "through_origin": False}
            },
    "Level": "Organ", # Organ or Voxel. Voxel level will provide 3-D dose-map, and also organ-level values.
    "Method": "S-Value", # S-value, or Monte-Carlo.
}


In [19]:
DoseCalculator = BaseDosimetry(config=config, 
                               nm_data=longSPECT, 
                               ct_data=longCTAC)
DoseCalculator.compute_tiac()
DoseCalculator.results

Database search function not implemented. Dosimetry for this patient might already exists...
Although mask for 1_Lesion is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for 2_Lesion_Hottest is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for 3_Lesion is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for 4_Lesion is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for 5_Lesion is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for Bladder_Experimental is present, we are ignoring it because this region was not included in the configuration input file.
Although mask for Femur_L is present, we are ignoring it because this region was not included in the configuration inpu

AssertionError: Only 1 data points available. Not enough points to perform fit.